# The demonstration of the tropical precipitation module on the different datasets

###  Loading the necessary packages

In [1]:
from dask_jobqueue import SLURMCluster # pip 
from dask.distributed import Client, progress 
import dask

import re
import matplotlib as mpl
# Define Agg as Backend for matplotlib when no X server is running
mpl.use('Agg')
import socket
import os
import importlib

import inspect
import timeit
import sys

### Function, which reads the status of the user in the slurm queue 

In [ ]:
#!pip install aqua

In [2]:
def squeue_user(username = "$USER"):
    _squeue_user = os.system("squeue --user="+str(username))
    return _squeue_user 

### Storing the path to the current repository into the variable

In [3]:
with os.popen("pwd ") as f:
    _pwd = f.readline()

pwd = re.split(r'[\n]', _pwd)[0]

### Setting the slurm job

In [266]:

extra_args=[
    "--error="+str(pwd)+"/slurm/logs/dask-worker-%j.err",
    "--output="+str(pwd)+"/slurm/output/dask-worker-%j.out"
]

cluster = SLURMCluster(
    name='dask-cluster', 
    cores=256,    
    memory="500 GB", 
    project="bb1153",
    queue= "compute", 
    walltime='04:30:50',
    job_extra=extra_args,
)
client = Client(cluster)
print(cluster.job_script())

#!/usr/bin/env bash

#SBATCH -J dask-worker
#SBATCH -p compute
#SBATCH -A bb1153
#SBATCH -n 1
#SBATCH --cpus-per-task=256
#SBATCH --mem=466G
#SBATCH -t 04:30:50
#SBATCH --error=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/logs/dask-worker-%j.err
#SBATCH --output=/work/bb1153/b382267/AQUA/diagnostics/tropical_rainfall/notebooks/slurm/output/dask-worker-%j.out

/home/b/b382267/mambaforge/envs/tropical-rainfall/bin/python -m distributed.cli.dask_worker tcp://136.172.124.3:39067 --nthreads 16 --nworkers 16 --memory-limit 29.10GiB --name dummy-name --nanny --death-timeout 60



/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/core.py:255: FutureWarning: job_extra has been renamed to job_extra_directives. You are still using it (even if only set to []; please also check config files). If you did not set job_extra_directives yet, job_extra will be respected for now, but it will be removed in a future release. If you already set job_extra_directives, job_extra is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/envs/tropical-rainfall/lib/python3.10/site-packages/dask_jobqueue/slurm.py:49: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/home/b/b382267/mambaforge/e

### Loading the slurm job to the queue 

In [267]:
cluster.scale(jobs=1)

### Checking our slurm job in the queue 

In [271]:
squeue_user()

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
           4589766   compute dask-wor  b382267  R       6:42      1 l40205


0

### Canceling the job if it is necessary. Set the job ID manually

In [7]:
Job_ID =   4586525
os.system("scancel " +str(Job_ID)) 

0

### Setting the path to the diagnostic repository 

In [11]:
sys.path.append(str(pwd)+'/../')
import src.shared_func
import src.tr_pr_mod

### Loading the extra function, which can be used in any diagnostic

In [281]:
while True:
    try:
        importlib.reload(src.shared_func)
        from  src.shared_func import time_interpreter,  animation_creator, image_creator,  xarray_attribute_update,  data_size
        break
    except NameError and AttributeError:
        import src.shared_func
        from  src.shared_func import time_interpreter,   animation_creator, image_creator,  xarray_attribute_update,  data_size
        break

### Loading the tropical precipitation module

In [253]:
while True:
    try:
        importlib.reload(src.tr_pr_mod)
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break
    except NameError and AttributeError:
        import src.tr_pr_mod
        from  src.tr_pr_mod import TR_PR_Diagnostic
        break

In [15]:
configdir = '../../../config/'
diagname  = 'tr_pr'
machine   = 'levante'

### Setting the class attributes

In [254]:
diag = TR_PR_Diagnostic()

diag.num_of_bins = 15
diag.first_edge = 0
diag.width_of_bin = 1*10**(-4)/diag.num_of_bins

last_edge = diag.first_edge  + diag.num_of_bins*diag.width_of_bin

### Importing the aqua module 

In [17]:
import aqua

In [18]:
from aqua import Reader
from aqua.reader import catalogue

In [19]:
catalogue(configdir=configdir)

IFS	tco3999-ng5	2.8km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- interpolated_sp_ci	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco2559-ng5	4km experiment, coupled with FESOM
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- interpolated_global	
	- interpolated_np	
	- interpolated_sp	
	- lra-r100-day	
	- lra-r100-mon	
IFS	tco1279-orca025	9km baseline, coupled to NEMO, deep conv ON
	- ICMGG_atm2d	
	- ICMU_atm2d	
	- ICMU_atm3d	
	- lra-r100-day	
	- lra-r100-mon	
IFS	test-tco2559	4km experiment, coupled with FESOM
	- ICMGG_atm2d	2d output
	- ICMU_atm3d	3d output

FESOM	tco3999-ng5	2.5km experiment, coupuled with IFS
	- elem_grid	
	- node_grid	
	- np	nearest-neighbor interpolation to lat-lon grid
	- interpolated_global2d	
	- interpolated_global_TS	
	- interpolated_global_UV	
	- interpolated_np	
	- interpolated_sp	
	- original_2d	original 2d output
	- original_3d	original 3d output
FESOM	tco2559-ng

levante:
  args:
    path: ../../../config//levante/catalog.yaml
  description: ''
  driver: intake.catalog.local.YAMLFileCatalog
  metadata: {}


# ICON data

In [20]:
reader = Reader(model="ICON", exp="ngc2009",  configdir=configdir, source="atm_2d_ml_R02B09", regrid="r200")
ICON_2009_reg = reader.retrieve()

In [21]:
ICON_2009_reg

<xarray.Dataset>
Dimensions:   (time: 37009, cell: 20971520)
Coordinates:
  * time      (time) datetime64[ns] 2020-01-20 ... 2022-03-01
Dimensions without coordinates: cell
Data variables: (12/33)
    cfh_lnd   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tciw      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    tclw      (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    cptgzvi   (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mslhfl    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msshfl    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    ...        ...
    10u       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    10v       (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msntrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mtntrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    msnsrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
    mtnsrf    (time, cell) float32 dask.array<chunksize=(6, 1048576), meta=np.ndarray>
Attributes:
    CDI:                  Climate Data Interface version 1.8.3rc (http://mpim...
    Conventions:          CF-1.6
    comment:              Sapphire Dyamond (k203123) on l40687 (Linux 4.18.0-...
    grid_file_uri:        http://icon-downloads.mpimet.mpg.de/grids/public/mp...
    history:              ./icon at 20220512 1522142023-04-03 00:51:05 retrie...
    institution:          Max Planck Institute for Meteorology/Deutscher Wett...
    number_of_grid_used:  15
    references:           see MPIM/DWD publications
    source:               git@gitlab.dkrz.de:icon/icon-aes.git@87a1eaded69e87...
    title:                ICON simulation
    uuidOfHGrid:          0f1e7d66-637e-11e8-913b-51232bb4d8f9

### size of entire dataset

In [ ]:
data_size(ICON_2009_reg)

### timestep of dataset is

In [275]:
time_interpreter(ICON_2009_reg)

'30m'

### Selecting the small part of dataset for tests

In [284]:
ICON_2009_small = ICON_2009_reg['tprate'][10:11,:]

### size of selected chunk

In [286]:
data_size(ICON_2009_small)

20971520

### Attribute update

In [285]:
ICON_2009_small = xarray_attribute_update(ICON_2009_small, ICON_2009_reg)

### Load selected chunk into a memory

In [ ]:
ICON_2009_small = ICON_2009_small.compute()

In [42]:
ICON_2009_small

<xarray.DataArray 'tprate' (time: 1, cell: 20971520)>
dask.array<getitem, shape=(1, 20971520), dtype=float32, chunksize=(1, 1048576), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2020-01-20T05:00:00
Dimensions without coordinates: cell
Attributes: (12/16)
    CDI_grid_type:                unstructured
    cell_methods:                 t: mean
    long_name:                    Mean total precipitation rate
    number_of_grid_in_reference:  1
    param:                        52.1.0
    standard_name:                pr
    ...                           ...
    shortName:                    tprate
    target_units:                 m s**-1
    factor:                       0.001
    offset:                       0
    src_units:                    kg m-2 s-1
    units_fixed:                  1

### Units of precepitation rate

In [287]:
ICON_2009_small.attrs['units']

'm s**-1'

In [277]:
time_interpreter(ICON_2009_small) 

'False. Load more timesteps then one'

### Name of coordinates

In [288]:
diag.coordinate_names(ICON_2009_small)

(None, None)

### Fastest histogram

In [289]:
hist_fast_ICON  = diag.hist1d_fast(ICON_2009_small,  preprocess = False)
hist_fast_ICON

<xarray.DataArray (bin: 15)>
array([2.0958727e+07, 8.4030000e+03, 2.5020000e+03, 1.0330000e+03,
       4.4300000e+02, 2.0900000e+02, 1.0700000e+02, 4.9000000e+01,
       3.1000000e+01, 8.0000000e+00, 6.0000000e+00, 1.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00])
Coordinates:
  * bin      (bin) float64 0.0 6.667e-06 1.333e-05 ... 8e-05 8.667e-05 9.333e-05
Attributes: (12/26)
    CDI_grid_type:                unstructured
    cell_methods:                 t: mean
    long_name:                    Mean total precipitation rate
    number_of_grid_in_reference:  1
    param:                        52.1.0
    standard_name:                pr
    ...                           ...
    institution:                  Max Planck Institute for Meteorology/Deutsc...
    number_of_grid_used:          15
    references:                   see MPIM/DWD publications
    source:                       git@gitlab.dkrz.de:icon/icon-aes.git@87a1ea...
    title:                        ICON simulation
    uuidOfHGrid:                  0f1e7d66-637e-11e8-913b-51232bb4d8f9

### Plot of obtained histogram 

In [293]:
diag.hist_plot(hist_fast_ICON, pdf = True, smooth = True, plot_title = "ICON, precipitation rate for one timestep")

## Plot of mean and median value of precipitation rate

In [274]:
diag.mean_and_median_plot(ICON_2009_small, savelabel = 'ICON')

#  MSWEP 

In [29]:
reader = Reader(model="MSWEP", exp="past", source="monthly",configdir=configdir)
#data = reader.retrieve(fix=False)
#data = reader.retrieve(apply_unit_fix=False)
MSWEP = reader.retrieve()

In [32]:
MSWEP 

<xarray.Dataset>
Dimensions:  (lon: 3600, lat: 1800, time: 502)
Coordinates:
  * lon      (lon) float32 -179.9 -179.8 -179.8 -179.6 ... 179.8 179.9 179.9
  * lat      (lat) float32 89.95 89.85 89.75 89.65 ... -89.75 -89.85 -89.95
  * time     (time) datetime64[ns] 1979-02-01 1979-03-01 ... 2020-11-01
Data variables:
    tprate   (time, lat, lon) float32 dask.array<chunksize=(1, 1800, 3600), meta=np.ndarray>
Attributes:
    history:  Created on 2021-02-04 03:412023-03-31 01:46:33 coordinates adju...

In [33]:


MSWEP_small = MSWEP['tprate'][0:2,:]
MSWEP_small = MSWEP_small.compute()

In [66]:
diag.time_interpreter_b(MSWEP)

diag.time_interpreter_b(MSWEP_small) 

timestep is month
timestep is month


'M'

In [148]:
hist_fast_MSWEP,  size_MSWEP,   del_time_fast_MSWEP  = diag.hist1d_fast(MSWEP_small)
hist_fast_MSWEP

<xarray.DataArray (bin: 15)>
array([1.3122303e+07, 1.2379830e+06, 3.5975000e+04, 3.1350000e+03,
       4.5500000e+02, 8.1000000e+01, 2.6000000e+01, 2.8000000e+01,
       6.0000000e+00, 1.0000000e+00, 4.0000000e+00, 2.0000000e+00,
       1.0000000e+00, 0.0000000e+00, 0.0000000e+00])
Coordinates:
  * bin      (bin) float64 0.0 0.0001333 0.0002667 ... 0.0016 0.001733 0.001867

In [152]:

fig = plt.figure()
ax = fig.add_subplot(111)

_pdf = True

diag.hist_plot(hist_fast_MSWEP,           pdf=_pdf,   color = 'tab:red',     label='fast')

ax.grid()

plt.savefig("./figures/pdf_hists_MSWEP.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

hist_plot function in the process 



In [64]:
fig = plt.figure()
plt.plot(diag.mean_per_timestep(MSWEP_small), label= 'mean')
plt.plot(diag.median_per_timestep(MSWEP_small), label='median')

plt.yscale('log')
plt.xlabel('Time', fontsize=12)
plt.ylabel('Precipitation per day', fontsize=12)
plt.title('Mean/median values of precipitation', fontsize =16)
plt.legend(fontsize=12)

#plt.yscale('log')
print('gmean ....')


plt.savefig("./figures/mean_and_median_MSWEP.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')

gmean ....


### Animation

In [ ]:
diag.animated(MSWEP_small,   label = 'MSWEP') 

In [173]:

scale_factor = 90/diag.trop_lat
fig = plt.figure(figsize=(8*scale_factor,6))
diag.ds_per_lat_range(MSWEP)['pr'][0,:,:].plot()#vmin=0, vmax=100)


plt.savefig("./figures/trop_rains_MSWEP.png",
            bbox_inches ="tight",
            pad_inches = 1,
            transparent = True,
            facecolor ="w",
            edgecolor ='w',
            orientation ='landscape')